In [1]:
# Dependencies
# ----------------------------------
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData
Base = declarative_base()

In [2]:
from sqlalchemy import Column, Integer, String, Float

In [3]:
# Create Shark Attack Classes
# ----------------------------------
class Shark_Attacks (Base):
    __tablename__ = 'shark_attacks'
    year = Column(Integer, primary_key=True)
    type = Column(String(255))
    country = Column(String(255))
    area = Column(String(255))
    activity = Column(String(255))
    fatal = Column(String(1))
    species = Column(String(255))

In [4]:
# Create Database Connection
database_path = "sharkattacks_db"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [5]:
import pandas as pd
csv_path = "./filtered_shark_attacks_v2.csv"
df = pd.read_csv("./filtered_shark_attacks_v2.csv")

new_df = df.rename(columns={"Fatal (Y/N)":"Fatal"})
clean_new_df = new_df.drop(columns=['Unnamed: 7', 'Unnamed: 8'])


In [6]:
df

,Year,Type,Country,Area,Activity,Fatal,Species,Unnamed: 7,Unnamed: 8
0,2020.0,Unprovoked,USA,Maui,Stand-Up Paddle boarding,N,Tiger shark,NaN,NaN
1,2020.0,Provoked,BAHAMAS,Exumas,Floating,N,NaN,NaN,NaN
2,2020.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,N,"""A small shark""",NaN,NaN
3,2020.0,Unprovoked,NEW ZEALAND,Southland,Surfing,N,Broadnose seven gill shark?,NaN,NaN
4,2020.0,Unprovoked,USA,North Carolina,Surfing,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6457,0.0,Unprovoked,AUSTRALIA,Western Australia,Diving,Y,NaN,NaN,NaN
6458,0.0,Unprovoked,AUSTRALIA,Western Australia,Pearl diving,Y,NaN,NaN,NaN
6459,0.0,Unprovoked,USA,North Carolina,Swimming,Y,NaN,NaN,NaN
6460,0.0,Unprovoked,PANAMA,NaN,NaN,Y,NaN,NaN,NaN


In [7]:
clean_new_df

,Year,Type,Country,Area,Activity,Fatal,Species
0,2020.0,Unprovoked,USA,Maui,Stand-Up Paddle boarding,N,Tiger shark
1,2020.0,Provoked,BAHAMAS,Exumas,Floating,N,NaN
2,2020.0,Unprovoked,AUSTRALIA,New South Wales,Surfing,N,"""A small shark"""
3,2020.0,Unprovoked,NEW ZEALAND,Southland,Surfing,N,Broadnose seven gill shark?
4,2020.0,Unprovoked,USA,North Carolina,Surfing,N,NaN
...,...,...,...,...,...,...,...
6457,0.0,Unprovoked,AUSTRALIA,Western Australia,Diving,Y,NaN
6458,0.0,Unprovoked,AUSTRALIA,Western Australia,Pearl diving,Y,NaN
6459,0.0,Unprovoked,USA,North Carolina,Swimming,Y,NaN
6460,0.0,Unprovoked,PANAMA,NaN,NaN,Y,NaN


In [8]:
clean_new_df.to_sql('shark_attacks', con=engine,  if_exists='replace')

In [9]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [10]:
# Create a Session Object to Connect to DB
# ----------------------------------
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [11]:
from sqlalchemy import inspect
from sqlalchemy import Column
inspector = inspect(engine)


# #Get table information
# print(inspector.get_table_names())
# table = inspector.get_table_names()

# #Get column information
# print(inspector.get_columns('shark_attacks'))

In [12]:
m = MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(table.name)
    shark_attacks_table = table.name
    for column in table.c:
        print(column.name)
    

shark_attacks
index
Year
Type
Country
Area
Activity
Fatal
Species 


In [13]:
from sqlalchemy import select
table = m.tables['shark_attacks']

#Select * from shark_attacks
select_statement = select([table])
res = conn.execute(select_statement)
for row in res:
    print(row)

(0, 2020.0, 'Unprovoked', 'USA', 'Maui', 'Stand-Up Paddle boarding', 'N', 'Tiger shark')
(1, 2020.0, 'Provoked', 'BAHAMAS', 'Exumas', 'Floating', 'N', None)
(2, 2020.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Surfing', 'N', '"A small shark"')
(3, 2020.0, 'Unprovoked', 'NEW ZEALAND', 'Southland', 'Surfing', 'N', 'Broadnose seven gill shark?')
(4, 2020.0, 'Unprovoked', 'USA', 'North Carolina', 'Surfing', 'N', None)
(5, 2020.0, 'Unprovoked', 'THAILAND', 'Phang Nga Province', 'Swimming', 'N', None)
(6, 2020.0, 'Unprovoked', 'AUSTRALIA', 'Queensland', 'Swimming', 'N', 'Lemon shark')
(7, 2020.0, 'Unprovoked', 'AUSTRALIA', 'Western Australia', 'Scuba diving', 'Y', 'White shark')
(8, 2020.0, 'Unprovoked', 'NEW CALEDONIA', None, None, 'N', None)
(9, 2020.0, 'Questionable', 'AUSTRALIA', 'Western Australia', 'Swimming after being swept off rocks', 'Y', None)
(10, 2019.0, 'Unprovoked', 'AUSTRALIA', 'Queensland', 'Swimming', 'N', 'Shovelnose "shark" which is a ray, not a shark)')
(11, 2019.0

(782, 2013.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(783, 2013.0, 'Provoked', 'USA', 'Florida', None, 'N', 'Nurse shark')
(784, 2013.0, 'Unprovoked', 'BAHAMAS', None, None, 'N', None)
(785, 2013.0, 'Unprovoked', 'USA', 'Florida', 'Casting a net', 'N', "4' shark")
(786, 2013.0, 'Unprovoked', 'BAHAMAS', None, 'Spearfishing', 'N', 'Blacktip shark')
(787, 2013.0, 'Unprovoked', 'USA', 'California', 'Swimming', 'N', 'Salmon shark')
(788, 2013.0, 'Invalid', 'USA', 'California', 'Marathon swimming', None, 'No shark involvement')
(789, 2013.0, 'Provoked', 'FRANCE', 'Bay of Biscay', 'Longline fishing for sharks', 'N', "Mako shark, 5'")
(790, 2013.0, 'Unprovoked', 'USA', 'Florida', 'Boogie boarding', 'N', None)
(791, 2013.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Wrangling a shark', 'N', None)
(792, 2013.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', None)
(793, 2013.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', 'White shark')
(794, 2013.0, 'Unprovoked', 'USA',

(907, 2012.0, 'Unprovoked', 'USA', 'Alabama', 'Wading or swimming', 'N', None)
(908, 2012.0, 'Boating', 'AUSTRALIA', 'Western Australia', 'Fishing', 'N', '4.5 m shark')
(909, 2012.0, 'Unprovoked', 'USA', 'Florida', 'Free diving', 'N', "Lemon shark, 4' to 5'")
(910, 2012.0, 'Unprovoked', 'REUNION', 'Saint Leu', 'Surfing', 'N', 'Bull shark')
(911, 2012.0, 'Unprovoked', 'FRENCH POLYNESIA', 'Tuamotus', 'Spearfishing', 'N', 'Lemon shark, 2.5m to 3m')
(912, 2012.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', None)
(913, 2012.0, 'Unprovoked', 'AUSTRALIA', 'South Australia', 'Surfing', 'N', 'White shark or bronze whaler')
(914, 2012.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', None)
(915, 2012.0, 'Unprovoked', 'USA', 'Massachusetts', 'Body surfing', 'N', 'Thought to involve a white shark')
(916, 2012.0, 'Invalid', 'USA', 'North Carolina', None, None, 'Shark involvement not confirmed')
(917, 2012.0, 'Unprovoked', 'REUNION', 'Trois-Bassins', 'Surfing', 'Y', None)
(918, 2012.0, 'Inval

(1326, 2009.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Snorkeling', 'N', 'Dusky shark, 2m')
(1327, 2009.0, 'Unprovoked', 'AUSTRALIA', 'Tasmania', 'Surfing', 'N', 'White shark, 5m')
(1328, 2009.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Surfing', 'N', 'White shark, 3.5m')
(1329, 2009.0, 'Boat', 'NEW ZEALAND', 'North Island', 'Fishing', 'N', 'Mako shark, 3m')
(1330, 2009.0, 'Unprovoked', 'ECUADOR', 'Galapagos Islands', 'Surfing', 'N', None)
(1331, 2009.0, 'Unprovoked', 'NEW ZEALAND', 'North Island', 'Swimming', 'N', 'Broadnose sevengill shark')
(1332, 2009.0, 'Unprovoked', 'CUBA', 'Guantanamo Province', 'Spearfishing', 'N', 'Bull shark')
(1333, 2008.0, 'Invalid', 'AUSTRALIA', 'Western Australia', 'Crabbing', None, 'No shark involvement')
(1334, 2008.0, 'Boating', 'AUSTRALIA', 'Tasmania', 'Yacht race', 'N', '2 m shark')
(1335, 2008.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Boogie Boarding', 'N', None)
(1336, 2008.0, 'Boating', 'AUSTRALIA', 'New South Wales', 'Kayakin

(1719, 2005.0, 'Unprovoked', 'USA', 'Florida', 'Wading', 'N', None)
(1720, 2005.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(1721, 2005.0, 'Invalid', 'USA', 'Florida', 'Surfing', None, 'Shark involvement not confirmed')
(1722, 2005.0, 'Invalid', 'LIBERIA', None, 'Swimming', None, 'Shark involvement prior to death was not confirmed')
(1723, 2005.0, 'Provoked', 'CHINA', 'Shanghai', 'Scuba diving in aquarium tank', 'N', "3.5 m [11.5']shark")
(1724, 2005.0, 'Invalid', 'AUSTRALIA', None, None, None, 'No shark involvement')
(1725, 2005.0, 'Unprovoked', 'USA', 'North Carolina', 'Swimming', 'N', "[4.5' to 5'] shark")
(1726, 2005.0, 'Unprovoked', 'USA', 'Texas', 'Holding onto an inflatable boat', 'N', "[4' to 5']")
(1727, 2005.0, 'Unprovoked', 'USA', 'Florida', 'Standing', 'N', None)
(1728, 2005.0, 'Unprovoked', 'USA', 'Florida', 'Fishing', 'N', 'Bull shark')
(1729, 2005.0, 'Unprovoked', 'USA', 'Florida', 'Swimming with boogie board', 'Y', "1.8 m [6'] bull shark")
(1730, 2005.0, 'U

...

(5182, 1931.0, 'Invalid', 'AUSTRALIA', 'New South Wales', 'Swimming', None, 'Shark involvement prior to death unconfirmed')
(5183, 1931.0, 'Unprovoked', 'AUSTRALIA', 'Torres Strait', None, 'N', None)
(5184, 1931.0, 'Unprovoked', 'AUSTRALIA', 'Torres Strait', 'Pearl diving', 'N', None)
(5185, 1931.0, 'Unprovoked', 'AUSTRALIA', 'Queensland', 'Swimming', 'N', "1.5 m [5'] shark")
(5186, 1931.0, 'Unprovoked', 'TONGA', 'Niua', 'Swimming, carrying tin can with mail to steamer', 'Y', None)
(5187, 1931.0, 'Unprovoked', 'PAPUA NEW GUINEA', None, None, 'UNKNOWN', None)
(5188, 1930.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Swimming', 'N', None)
(5189, 1930.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Fell overboard', 'Y', None)
(5190, 1930.0, 'Watercraft', 'AUSTRALIA', 'Victoria', 'Fishing', 'N', "7 shark's teeth found embedded in the woodwork of the boat")
(5191, 1930.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', None, 'Y', None)
(5192, 1930.0, 'Unprovoked', 'AUSTRALIA', 'Queensla

(5584, 1909.0, 'Invalid', 'ITALY', 'Sicily', 'On December 28, 1908, an earthquake, followed by tsunamis, destroyed coastal towns in Silcily and southern Italy, killing more than 100,000 people', None, 'Shark involvement prior to death was not confirmed')
(5585, 1909.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', None, 'N', None)
(5586, 1908.0, 'Unprovoked', 'SOUTH AFRICA', 'Western Cape Province', 'Swimming', 'Y', None)
(5587, 1908.0, 'Unprovoked', 'MEXICO', 'Quintana Roo', 'Swimming ashore from capsized boat', 'Y', None)
(5588, 1908.0, 'Unprovoked', 'AUSTRALIA', 'Torres Strait', 'Fell from the jetty', 'Y', 'Remains recovered from 3 sharks')
(5589, 1908.0, 'Unprovoked', 'USA', 'California', 'On fishing boat & trailing hand in the water', 'N', None)
(5590, 1908.0, 'Unprovoked', 'SPAIN', 'Galicia', 'Fell overboard from P&O steamship Arabia', 'Y', None)
(5591, 1908.0, 'Invalid', 'SPAIN', 'Canary Islands', None, None, 'Questionable')
(5592, 1908.0, 'Unprovoked', 'CROATIA', 'Adriatic Sea'

(5798, 1895.0, 'Unprovoked', 'NEW ZEALAND', 'South Island', '"Boat accident"', 'Y', None)
(5799, 1895.0, 'Unprovoked', 'SOUTH AFRICA', 'Eastern Cape Province', '"Crossing the river"', 'Y', None)
(5800, 1895.0, 'Unprovoked', 'NEW ZEALAND', 'South Island', 'Swimming', 'N', None)
(5801, 1895.0, 'Unprovoked', 'INDONESIA', 'Moluccas', 'Swimming', 'Y', None)
(5802, 1895.0, 'Unprovoked', 'AUSTRALIA', 'Queensland', 'Swimming', 'N', None)
(5803, 1895.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Swimming', 'Y', None)
(5804, 1895.0, 'Unprovoked', 'PANAMA', 'Bocas del Toro', 'Swimming, after sailboat capsized', 'Y', None)
(5805, 1895.0, 'Watercraft', 'AUSTRALIA', 'Western Australia', None, 'N', None)
(5806, 1894.0, 'Provoked', 'USA', 'Florida', 'Fishing', 'N', "12' shark")
(5807, 1894.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Bathing', 'N', "10' to 12' shark")
(5808, 1894.0, 'Unprovoked', 'USA', 'Alabama', 'Swimming', 'Y', None)
(5809, 1894.0, 'Unprovoked', 'USA', 'Virginia', 'Bathing

(6083, 1868.0, 'Sea Disaster', 'VIETNAM', None, 'A junk foundered', 'Y', None)
(6084, 1868.0, 'Sea Disaster', 'AUSTRALIA', 'South Australia', 'Fishing', 'N', None)
(6085, 1868.0, 'Unprovoked', 'AUSTRALIA', 'Torres Strait', 'Swimming', 'Y', None)
(6086, 1868.0, 'Unprovoked', 'NEW ZEALAND', 'North Island', 'Bathing close inshore', 'N', '"Shark caught later"')
(6087, 1867.0, 'Unprovoked', 'EGYPT', None, 'Swimming', 'Y', None)
(6088, 1867.0, 'Unprovoked', 'MEXICO', 'Veracruz', 'boat from the Austrian ship Elizabeth', 'Y', None)
(6089, 1867.0, 'Sea Disaster', 'CUBA', 'Villa Clara Province', 'boat from ship Josephine capsized in squall', 'Y', None)
(6090, 1867.0, 'Watercraft', 'AUSTRALIA', 'Victoria', None, 'N', None)
(6091, 1867.0, 'Unprovoked', 'CUBA', 'Matanzas Province (north coast)', 'Painting a ship', 'Y', None)
(6092, 1866.0, 'Unprovoked', 'USA', 'Alabama', 'Bathing', 'Y', None)
(6093, 1866.0, 'Invalid', 'AUSTRALIA', 'Western Australia', 'Fell overboard', None, 'Shark involvement prio

(6203, 1847.0, 'Sea Disaster', 'AUSTRALIA', 'Queensland', 'Wreck of the Sovereign', 'N', None)
(6204, 1847.0, 'Sea Disaster', 'MEXICO', None, 'Wreck of the Tweed', 'Y', None)
(6205, 1847.0, 'Unprovoked', 'AUSTRALIA', 'Queensland', None, 'N', None)
(6206, 1847.0, 'Invalid', 'USA', 'South Carolina', 'Swimming', None, 'Shark involvement prior to death unconfirmed')
(6207, 1847.0, 'Unprovoked', 'USA', 'South Carolina', 'Watercraft', 'N', None)
(6208, 1846.0, 'Sea Disaster', 'MEXICO', 'Veracruz', 'Wreck of the USS Somers', 'Y', None)
(6209, 1846.0, 'Unprovoked', 'AFRICA', None, 'Jumped into river', 'Y', None)
(6210, 1846.0, 'Unprovoked', 'CUBA', 'Havana Province', 'Bathing', 'N', None)
(6211, 1845.0, 'Unprovoked', 'AUSTRALIA', 'New South Wales', 'Fishing', 'Y', None)
(6212, 1845.0, 'Unprovoked', 'MEXICO', 'Vera Cruz', 'Bathing', 'Y', None)
(6213, 1845.0, 'Unprovoked', 'ENGLAND', None, None, 'N', None)
(6214, 1845.0, 'Unprovoked', 'USA', 'Florida', 'Seine netting', 'Y', None)
(6215, 1845.0, 

In [14]:
#Select * from shark_attacks
select_statement = select([table]).where(table.c.Country == 'USA')
res = conn.execute(select_statement)
for row in res:
    print(row)

(0, 2020.0, 'Unprovoked', 'USA', 'Maui', 'Stand-Up Paddle boarding', 'N', 'Tiger shark')
(4, 2020.0, 'Unprovoked', 'USA', 'North Carolina', 'Surfing', 'N', None)
(15, 2019.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', None)
(19, 2019.0, 'Unprovoked', 'USA', 'Hawaii', 'Stand-Up Paddleboarding', 'N', "10'-12' tiger shark")
(20, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', 'Juvenile blacktip shark')
(26, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "4' to 5' blacktip shark")
(31, 2019.0, 'Provoked', 'USA', 'Florida', 'Surfing', 'N', "4' to 5' shark")
(34, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(36, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(37, 2019.0, 'Unprovoked', 'USA', 'Guam', 'Surfing', 'N', 'Juvenile shark')
(39, 2019.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', "6' to 7' shark")
(40, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(41, 2019.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N',

(614, 2015.0, 'Invalid', 'USA', 'Florida', 'Swimming after falling overboard', None, 'No shark involvement')
(617, 2015.0, 'Unprovoked', 'USA', 'Florida', None, 'N', '"A small shark"')
(622, 2014.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', "White shark, 8' to 10'")
(630, 2014.0, 'Boating', 'USA', 'California', None, 'N', 'White shark')
(631, 2014.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "2' shark")
(632, 2014.0, 'Unprovoked', 'USA', 'Hawaii', 'Snorkeling', 'N', '1.5 m shark')
(634, 2014.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(635, 2014.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', "Tiger shark, 12'")
(637, 2014.0, 'Unprovoked', 'USA', 'Hawaii', 'Stand-Up Paddleboarding', 'N', 'Tiger shark')
(638, 2014.0, 'Unprovoked', 'USA', 'Hawaii', 'Stand-Up Paddleboarding', 'N', None)
(639, 2014.0, 'Boating', 'USA', 'California', 'Canoeing', 'N', None)
(640, 2014.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', "12' to 14' shark")
(642, 2014.0, 'Unprovoked

(1621, 2006.0, 'Unprovoked', 'USA', 'Florida', 'Spearfishing', 'N', '1.8 m blacktip "reef" shark')
(1622, 2006.0, 'Unprovoked', 'USA', 'Florida', 'Body boarding', 'N', None)
(1623, 2006.0, 'Provoked', 'USA', 'Florida', 'Fishing', 'N', None)
(1625, 2006.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "Possibly a 1' to 3' blacktip or spinner shark")
(1627, 2006.0, 'Unprovoked', 'USA', 'California', 'Scuba diving', 'N', None)
(1628, 2006.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "2' to 3' shark")
(1629, 2006.0, 'Unprovoked', 'USA', 'South Carolina', 'Playing', 'N', "3' to 4' shark")
(1631, 2006.0, 'Unprovoked', 'USA', 'Hawaii', 'Spearfishing', 'N', "Tiger shark, 8' to 12'")
(1632, 2006.0, 'Provoked', 'USA', 'Hawaii', 'Tagging sharks', 'N', "Tiger shark, 15' female")
(1633, 2006.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing', 'N', None)
(1634, 2006.0, 'Provoked', 'USA', 'Hawaii', 'Spearfishing', 'N', "Grey reef shark, 5' to 8'")
(1636, 2006.0, 'Unprovoked', 'USA', 'Guam', 'Scuba d

(1951, 2003.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', 'Species unidentified')
(1964, 2002.0, 'Unprovoked', 'USA', 'California', 'Body boarding', 'N', "3 m to 4.5 m [10' to 15'] white shark")
(1965, 2002.0, 'Unprovoked', 'USA', 'Hawaii', 'Swimming', 'N', "2.4 m to 3 m [8' to 10'] grey colored shark")
(1967, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "6' shark")
(1968, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Boogie boarding', 'N', "3' to 4' shark")
(1970, 2002.0, 'Unprovoked', 'USA', 'Hawaii', 'Swimming', 'N', 'A “small” shark')
(1972, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(1973, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', 'Blacktip or spinner shark')
(1974, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "3.5' to 4.5' shark")
(1975, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "3.5' to 4' shark")
(1976, 2002.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "4' to 5' shark")
(1977, 2002.0, 'Unprovoked', 'USA

(2318, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(2320, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "1.8 m [6'] shark")
(2322, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "3' to 4' shark")
(2324, 1998.0, 'Unprovoked', 'USA', 'California', 'Boogie boarding', 'N', "White shark, 5 m to 6 m [16.5' to 20']")
(2325, 1998.0, 'Unprovoked', 'USA', 'Virginia', 'Spearfishing', 'N', "Dusky shark, 12'")
(2327, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Body-boarding', 'N', "4.5' to 5' shark")
(2332, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(2338, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(2343, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', "Lemon shark, 1.5 m [5'], identified by the surfer")
(2344, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', "1.8 m [6'] shark, possibly a blacktip")
(2345, 1998.0, 'Unprovoked', 'USA', 'Florida', 'Swimming / surfing', 'N', None)
(2348, 1998.0, 'Unprovoked', 'USA'

(2959, 1986.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', "Lemon shark, 1.8 m to 2.4 m [6' to 8'], tooth fragment recovered")
(2962, 1986.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', 'Bull shark')
(2963, 1986.0, 'Invalid', 'USA', 'Florida', None, None, 'Shark involvement prior to death was not confirmed')
(2964, 1986.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(2965, 1986.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', None)
(2966, 1986.0, 'Unprovoked', 'USA', 'North Carolina', 'Surfing', 'N', None)
(2971, 1986.0, 'Unprovoked', 'USA', 'South Carolina', None, 'N', "1.5 m [5'] shark")
(2975, 1986.0, 'Unprovoked', 'USA', 'Hawaii', 'Fishing, fell from rocks & disappeared', 'Y', None)
(2985, 1986.0, 'Unprovoked', 'USA', 'Florida', 'Surfing', 'N', 'Blacktip shark')
(2986, 1986.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', "1.5 m [5'] shark")
(2988, 1985.0, 'Unprovoked', 'USA', 'North Carolina', None, 'N', 'Sandtiger shark')
(2990, 1985.0, 'Boat', 'USA', 'C

(3408, 1974.0, 'Unprovoked', 'USA', 'California', 'Scuba diving (but on surface)', 'N', "White shark, 5 m to 6 m [16.5 to 20']")
(3410, 1974.0, 'Unprovoked', 'USA', 'Oregon', 'Surfing', 'N', 'White shark, 4.5 m')
(3416, 1974.0, 'Unprovoked', 'USA', 'California', 'Surfing', 'N', 'White shark, 5 m')
(3417, 1974.0, 'Unprovoked', 'USA', 'California', 'Free diving for abalone (submerged)', 'N', "White shark, 5.5 m [18']")
(3418, 1974.0, 'Unprovoked', 'USA', 'Georgia', 'Swimming', 'Y', "small sharks'")
(3419, 1974.0, 'Sea Disaster', 'USA', 'Florida', 'Adrift after the sinking of the motor yacht Princess Dianne', 'Y', "3.7 m [12'] sharks")
(3420, 1974.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(3421, 1974.0, 'Unprovoked', 'USA', 'California', 'Free diving (but on surface)', 'N', "White shark, 5 m [16.5']")
(3425, 1974.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'N', None)
(3443, 1973.0, 'Unprovoked', 'USA', 'Hawaii', 'Swimming', 'N', None)
(3446, 1973.0, 'Unprovoked', 'USA',

(4285, 1958.0, 'Unprovoked', 'USA', 'Hawaii', 'Surfing on air mattress', 'Y', "Tiger shark, 4.6 m to 7.6 m [15' to 25']")
(4290, 1958.0, 'Boat', 'USA', 'California', 'Fishing', 'N', 'White shark')
(4291, 1958.0, 'Unprovoked', 'USA', 'California', 'Swimming near jetty', 'N', None)
(4292, 1958.0, 'Unprovoked', 'USA', 'Florida', None, 'N', None)
(4297, 1958.0, 'Provoked', 'USA', 'New York', 'Spearfishing', 'N', None)
(4301, 1958.0, 'Unprovoked', 'USA', 'Florida', 'Swimming, ducking for shells in water 0.9 m deep', 'N', "Tiger shark, 1.5 m to 1.8 m [5' to 6']")
(4302, 1958.0, 'Unprovoked', 'USA', 'Florida', 'Swimming, ducking for shells in water 0.9 m deep', 'N', "Tiger shark, 1.5 m to 1.8 m [5' to 6']")
(4303, 1958.0, 'Unprovoked', 'USA', 'Florida', None, 'Y', None)
(4307, 1958.0, 'Provoked', 'USA', 'Florida', 'Free diving', 'N', "Nurse shark, 1.5 m [5'] identified by Dr. E. Clark on description of shark")
(4310, 1958.0, 'Unprovoked', 'USA', 'Florida', None, 'N', None)
(4311, 1958.0, 'Pro

(5813, 1894.0, 'Unprovoked', 'USA', 'Florida', 'Swimming / floating on his back', 'N', None)
(5815, 1894.0, 'Unprovoked', 'USA', 'Florida', 'Bathing', 'UNKNOWN', None)
(5817, 1894.0, 'Unprovoked', 'USA', 'North Carolina', 'Bathing', 'N', None)
(5837, 1892.0, 'Unprovoked', 'USA', 'Texas', None, 'N', None)
(5850, 1891.0, 'Provoked', 'USA', 'New Jersey', 'Shooting sharks', 'N', None)
(5851, 1891.0, 'Invalid', 'USA', 'Virginia', 'Fishing for sharks when he became entangled in net & fell overboard', None, 'Shark involvement prior to death unconfirmed')
(5857, 1890.0, 'Provoked', 'USA', 'New Jersey', 'Fishing for bluefish', 'N', '5\'7" shark')
(5858, 1890.0, 'Unprovoked', 'USA', 'Connecticut', 'Treading for clams', 'N', None)
(5873, 1889.0, 'Invalid', 'USA', 'Hawaii', 'Parachuted from balloon', None, 'Shark involvement prior to death unconfirmed')
(5875, 1889.0, 'Unprovoked', 'USA', 'Florida', 'Swimming', 'Y', None)
(5876, 1889.0, 'Unprovoked', 'USA', 'Texas', 'Fishing', 'N', "18' shark")
(5

In [15]:
#Save dataframe to an sqlite file
engine = create_engine('sqlite:///shark_attacks.db', echo=True)
sqlite_connection = engine.connect()


2020-10-19 21:57:11,590 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-19 21:57:11,591 INFO sqlalchemy.engine.base.Engine ()
2020-10-19 21:57:11,593 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-19 21:57:11,595 INFO sqlalchemy.engine.base.Engine ()


In [16]:
#Set variable to sql table name 
sqlite_table = "shark_attacks"
clean_new_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')



2020-10-19 21:57:11,618 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("shark_attacks")
2020-10-19 21:57:11,620 INFO sqlalchemy.engine.base.Engine ()
2020-10-19 21:57:11,622 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("shark_attacks")
2020-10-19 21:57:11,622 INFO sqlalchemy.engine.base.Engine ()
2020-10-19 21:57:11,626 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE shark_attacks (
	"index" BIGINT, 
	"Year" FLOAT, 
	"Type" TEXT, 
	"Country" TEXT, 
	"Area" TEXT, 
	"Activity" TEXT, 
	"Fatal" TEXT, 
	"Species " TEXT
)


2020-10-19 21:57:11,628 INFO sqlalchemy.engine.base.Engine ()
2020-10-19 21:57:11,632 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-19 21:57:11,633 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_shark_attacks_index ON shark_attacks ("index")
2020-10-19 21:57:11,634 INFO sqlalchemy.engine.base.Engine ()
2020-10-19 21:57:11,637 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-19 21:57:11,647 INFO sqlalchemy.engine.base.Engine BEGIN (implici

In [17]:
sqlite_connection.close()